In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Unzip dataset
import zipfile
dataset_zip = '/content/drive/MyDrive/yolo_dataset_degraded.zip'
!mkdir -p /content/datasetdegraded
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/datasetdegraded')

# Verify dataset structure
!ls /content/datasetdegraded
!tree -L 2 /content/datasetdegraded

# Install packages
!pip install ultralytics torch torchvision --quiet

# Verify installations
import ultralytics
import torch
print(f"Ultralytics version: {ultralytics.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
images	labels
/bin/bash: line 1: tree: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 866.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
dataset_yaml = """
train: /content/datasetdegraded/images/train
val: /content/datasetdegraded/images/val
nc: 3
names: ['bicycle', 'boat', 'car']
"""

with open('/content/datasetdegraded.yaml', 'w') as f:
    f.write(dataset_yaml)

from ultralytics import YOLO
import torch
import os  # Added for path verification

# Verify paths
assert os.path.exists('/content/datasetdegraded/images/train'), "Train path not found!"
assert os.path.exists('/content/datasetdegraded/images/val'), "Val path not found!"

# GPU Check
print(f"GPU: {torch.cuda.get_device_name(0)}")
model = YOLO('yolov8n.pt')

results = model.train(
    data='/content/datasetdegraded.yaml',
    epochs=100,
    patience=5,
    imgsz=640,
    batch=16,
    device=0,
    name='bw_vehicle_detector',
    optimizer='Adam',
    lr0=0.005,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.1,
    degrees=5.0,
    fliplr=0.3,
    mosaic=0.0,
    mixup=0.0,
    copy_paste=0.0,
    seed=42
)

# Validate
metrics = model.val()
print(f"mAP50-95: {metrics.box.map:.4f}")

# Export and verify
best_model_path = '/content/runs/detect/bw_vehicle_detector/weights/best.pt'
model.export(format='onnx')
onnx_path = best_model_path.replace('.pt', '.onnx')

loaded_model = YOLO(best_model_path)
print("\nTest inference on validation set:")
_ = loaded_model.predict('/content/datasetdegraded/images/val', save=True, imgsz=640)  # Fixed path

print(f"\nTraining complete. Models saved:")
print(f"- PyTorch: {best_model_path}")
print(f"- ONNX: {onnx_path}")

GPU: Tesla T4


100%|██████████| 6.25M/6.25M [00:00<00:00, 299MB/s]


Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasetdegraded.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.3, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.1, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=bw_vehicle_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, 

100%|██████████| 755k/755k [00:00<00:00, 95.0MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        507        933      0.625      0.418      0.474      0.281
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 47, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

52 epochs completed in 0.513 hours.
Optimizer stripped from runs/detect/bw_vehicle_detector/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/bw_vehicle_detector/weights/best.pt, 6.2MB

Validating runs/detect/bw_vehicle_detector/weights/best.pt...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:06<00:00,  2.36it/s]


                   all        507        933       0.68      0.417      0.487      0.282
               bicycle        153        206      0.515      0.639      0.627      0.397
                  boat        113        244      0.736      0.209      0.317      0.131
                   car        265        483      0.789      0.403      0.518      0.317
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/bw_vehicle_detector
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2867.1±921.9 MB/s, size: 98.0 KB)


val: Scanning /content/datasetdegraded/labels/val.cache... 507 images, 0 backgrounds, 0 corrupt: 100%|██████████| 507/507 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:06<00:00,  4.73it/s]


                   all        507        933      0.684      0.415      0.488      0.283
               bicycle        153        206      0.514      0.636      0.628      0.398
                  boat        113        244      0.748      0.206      0.316      0.132
                   car        265        483      0.789      0.402      0.519      0.317
Speed: 0.4ms preprocess, 4.9ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/bw_vehicle_detector2
mAP50-95: 0.2827
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from 'runs/detect/bw_vehicle_detector/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.9 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.56', 'onnxruntime-gpu'] not found, attempti